#This notebook Contains the code required for testing any random audio sample(s)

#Loading Libraries and Dataset

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
import librosa.display
import os

In [2]:
import librosa as lb

###Testing multiple files:
create a dataset with a column name as filenames and load it into google drive

In [3]:
import sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Loading Dataset, this dataset should contain names of the files under a column filename.
files_df=pd.read_csv("/content/drive/MyDrive/Mosaic23/file_info.csv")

In [5]:
files_df.head()

,start,end,crackles,weezels,pid,mode,filename
0,1.5629,3.4694,0.0,1.0,195,sc,195_1b1_Ar_sc_Litt3200_0.wav
1,3.4694,5.4677,0.0,0.0,195,sc,195_1b1_Ar_sc_Litt3200_1.wav
2,5.4677,7.5242,0.0,1.0,195,sc,195_1b1_Ar_sc_Litt3200_2.wav
3,7.5242,9.8468,0.0,0.0,195,sc,195_1b1_Ar_sc_Litt3200_3.wav
4,9.8468,11.1340,0.0,0.0,195,sc,195_1b1_Ar_sc_Litt3200_4.wav


#Feature Extraction

In [6]:
#function to get various features : MFCC,STFt,MelSpectrogram
def getFeatures(path):
    soundArr,sample_rate=lb.load(path)
    mfcc=lb.feature.mfcc(y=soundArr,sr=sample_rate)
    cstft=lb.feature.chroma_stft(y=soundArr,sr=sample_rate)
    mSpec=lb.feature.melspectrogram(y=soundArr,sr=sample_rate)

    return mfcc,cstft,mSpec

###Run if dealing with muliple files

In [ ]:
#Creating Feature array for each different transform for testing set
root='/content/drive/MyDrive/Mosaic23/processed_audio_files/'
mfcc,cstft,mSpec=[],[],[]

for idx,row in files_df.iterrows():
    path=root + row['filename']
    a,b,c=getFeatures(path)
    mfcc.append(a)
    cstft.append(b)
    mSpec.append(c)
    
mfcc_test=np.array(mfcc)
cstft_test=np.array(cstft)
mSpec_test=np.array(mSpec)

/usr/local/lib/python3.9/dist-packages/librosa/core/pitch.py:102: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


#Loading Model

In [7]:
reconstructed_model = keras.models.load_model("/content/drive/MyDrive/Mosaic23/models/MosaicModel.h5")

In [8]:
#if you want to check a single audio file
path='/content/drive/MyDrive/Mosaic23/processed_audio_files/195_1b1_Ar_sc_Litt3200_0.wav'
mfcc,cstft,mSpec,tempogram=[],[],[],[]


a,b,c=getFeatures(path)
mfcc.append(a)
cstft.append(b)
mSpec.append(c)
    
mfcc_test=np.array(mfcc)
cstft_test=np.array(cstft)
mSpec_test=np.array(mSpec)

In [9]:
predictions=reconstructed_model.predict({"mfcc":mfcc_test,"croma":cstft_test,"mspec":mSpec_test})

1/1 [==============================] - 9s 9s/step


In [10]:
classes = np.argmax(predictions, axis = 1)
print(classes)

[3]


In [11]:
def return_output(classes):
  if(classes[0]==0) :
    return 'both'
  if(classes[0]==1) :
    return 'crackles'
  if(classes[0]==2):
     return 'normal'
  if(classes[0]==3) :
    return 'weezels'

In [12]:
print(return_output(classes))

weezels
